In [1]:
import pandas as pd
from collections import OrderedDict
from lib.labm8 import fs

import analyze
import db
from db import *
%run util.py

hostname = "cc1"
db.init(hostname)

'mysql://cc1:3306/project_b'

In [8]:
def comment(*msg, prefix=''):
    return '\n'.join(f'// {prefix}{line}' for line in " ".join(msg).strip().split('\n'))

# Build failures

In [ ]:
datadir = "outbox/CLSmithResults/bf"
fs.mkdir(datadir)

with Session(commit=False) as s:
    q = s.query(CLSmithResult).filter(CLSmithResult.classification == "bf")
    for result in q:
        vendor = vendor_str(result.testbed.platform)
        reportpath = f"{datadir}/kernel-{vendor}-{result.id}.cl"
        print(reportpath)
        with open(reportpath, "w") as outfile:
            print(comment("stderr:"), file=outfile)
            print(comment(result.stderr, prefix='  '), file=outfile)
            print(result.program.src, file=outfile)

# Wrong-code bugs

In [11]:
datadir = "outbox/CLSmithResults/w"
fs.mkdir(datadir)


with Session(commit=False) as s:
    q = s.query(CLSmithResult).filter(CLSmithResult.classification == "w")
    for result in q:
        vendor = vendor_str(result.testbed.platform)
        reportpath = f"{datadir}/kernel-{vendor}-{result.id}.cl"
        
        # get expected output
        majority_output = analyze.get_majority_output(s, CLSMITH_TABLES, result)
        assert result.stdout != majority_output  # sanity check

        # use reduced src if possible
        reduction = s.query(CLSmithReduction).filter(CLSmithReduction.id == result.id).first()
        src = result.program.src
        # src = reduction.src if reduction else result.program.src

        print(reportpath)
        with open(reportpath, "w") as outfile:
            print(comment("Test case minimized?", "yes" if reduction else "no"), file=outfile)
            print(comment(''), file=outfile)
            print(comment("stderr:"), file=outfile)
            print(comment(result.stderr, prefix='  '), file=outfile)
            print(comment("output:"), file=outfile)
            print(comment(result.stdout, prefix='  '), file=outfile)
            print(comment("expected output:"), file=outfile)
            print(comment(majority_output, prefix='  '), file=outfile)
            print(src, file=outfile)
        break

outbox/CLSmithResults/w/kernel-nvidia-4321.cl
